In [6]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

In [7]:
env = make_vec_env("G1GoalDist-v1", 16)

In [8]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    learning_rate=1e-4,
    # learning_starts=1000,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [9]:
import copy

def soft_update(task_model, meta_model, tau=0.2):
    for target_param, local_param in zip(meta_model.parameters(), task_model.parameters()):
        target_param.data.copy_(target_param.data + tau * (local_param.data-target_param.data))

def reptile(n_reptile_iterations=100000, n_sac_iterations=50000):
    
    meta_policy = copy.copy(model.policy)
    tasks = model.env.env_method('sample_tasks', (n_reptile_iterations), indices=0)[0]

    for task in tasks:

        print("Task: ", task)
                
        model.env.env_method('reset_task', (task))
        model.policy = copy.copy(meta_policy)
        # model.replay_buffer.reset()

        model.learn(
            total_timesteps=n_sac_iterations, 
            log_interval=100,
            tb_log_name='reptile',
            reset_num_timesteps=False,
        )

        soft_update(model.policy, meta_policy, 0.2)


In [10]:
reptile()

Task:  {'direction': 4.39669994022792, 'orienation': 4.280881071181969}
Logging to ./logs/sac/reptile_0


NameError: name '__dir__' is not defined

In [12]:
env = gym.make("G1GoalDist-v1")

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


<_MjDataBodyViews
  cacc: array([0., 0., 0., 0., 0., 0.])
  cfrc_ext: array([0., 0., 0., 0., 0., 0.])
  cfrc_int: array([0., 0., 0., 0., 0., 0.])
  cinert: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  crb: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  cvel: array([0., 0., 0., 0., 0., 0.])
  subtree_angmom: array([0., 0., 0.])
  subtree_com: array([0., 0., 0.])
  subtree_linvel: array([0., 0., 0.])
  xfrc_applied: array([0., 0., 0., 0., 0., 0.])
  ximat: array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
  xipos: array([0., 0., 0.])
  xmat: array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
  xpos: array([0., 0., 0.])
  xquat: array([0., 0., 0., 0.])
>